In [1]:
import os

In [2]:
os.chdir("../")

In [18]:
%pwd

'C:\\Documents\\chicken-disease\\Chicken-Disease-classification\\src'

In [9]:
os.chdir(r"C:\Documents\chicken-disease\Chicken-Disease-classification\src")


In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [ ]:
import sys
sys.path.append(r"C:\Documents\chicken-disease\Chicken-Disease-classification\src")
from cnnClassifier.constants import *

from cnnClassifier.utils.common import read_yaml, create_directories

In [10]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        print("h11")
        self.config = read_yaml(config_filepath)
        print("h22")
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config

In [14]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [15]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config


    
    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )

        self.save_model(path=self.config.base_model_path, model=self.model)


    
    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model
    

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


In [19]:
try:
    print("Initializing ConfigurationManager...")
    config = ConfigurationManager()
    
    print("Getting base model configuration...")
    prepare_base_model_config = config.get_prepare_base_model_config()
    print(f"Configuration: {prepare_base_model_config}")
    
    print("Initializing PrepareBaseModel...")
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    
    print("Fetching base model...")
    prepare_base_model.get_base_model()
    
    print("Updating base model...")
    prepare_base_model.update_base_model()
    
except Exception as e:
    print(f"An error occurred: {str(e)}")
    raise e


Initializing ConfigurationManager...
An error occurred: Error reading YAML file config\config.yaml: [Errno 2] No such file or directory: 'config\\config.yaml'
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Documents\chicken-disease\Chicken-Disease-classification\src\cnnClassifier\utils\common.py", line 30, in read_yaml
    with open(path_to_yaml, "r") as yaml_file:
FileNotFoundError: [Errno 2] No such file or directory: 'config\\config.yaml'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\ImadEddineCharcha\anaconda3\envs\chicken\lib\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\ImadEddineCharcha\AppData\Local\Temp\ipykernel_27292\924294805.py", line 20, in <module>
    raise e
  File "C:\Users\ImadEddineCharcha\AppData\Local\Temp\ipykernel_27292\924294805.py", line 3, in <module>
    config = ConfigurationManager()
  File "C:\Users\ImadEddineCharcha\AppData\Local\Temp\ipykernel_27292\1617110292.py", line 6, in __init__
    self.config = read_yaml(config_filepath)
  File "c: